In [1]:
import dag_vis.dag_vis as dag_vis
import random
from parsl import *
import json

workers = ThreadPoolExecutor(max_workers=4)
dfk = DataFlowKernel(executors=[workers])

In [2]:
import time    
@App('python', dfk)
def sleep_fail(sleep_dur, sleep_rand_max, fail_prob, inputs=[]):
    import time
    import random

    s = sleep_dur + random.randint(-sleep_rand_max, sleep_rand_max)
    print("Sleeping for : ", s)
    time.sleep(s)
    x = float(random.randint(0,100)) / 100
    if x <= fail_prob :
        print("Fail")
        raise Exception("App failure")
    else:
        print("Succeed")

@App('python', dfk)
def test1 (numtasks ) :
    fus = []
    for i in range(0,10):

        fu = sleep_fail(0, 0, .5)
        fus.extend([fu])


    count = 0
    for fu in fus :
        try:
            x = fu.result()
        except Exception as e:
            count += 1
            print("Caught fail ")

    print("Caught failures of  {0}/{1}".format(count, len(fus)))

@App('python', dfk)
def test_deps (numtasks) :

    '''
    App1   App2  ... AppN
    '''
    fus = []
    for i in range(0,10):
        fu = sleep_fail(1, 0, .9)
        fus.extend([fu])

    '''
    App1   App2  ... AppN
    |       |        |
    V       V        V
    App1   App2  ... AppN
    '''

    fus_2 = []
    for fu in fus:
        fu = sleep_fail(0, 0, .5, inputs=[fu])
        fus_2.extend([fu])

    '''
    App1   App2  ... AppN
      |       |        |
      V       V        V
    App1   App2  ... AppN
       \      |       /
        \     |      /
          App_Final
    '''
    fu_final = sleep_fail(1, 0, 0, inputs=fus_2)


    print("Final status : ", fu_final.result())
    time.sleep(1)

In [3]:
test_deps(1000)

<AppFuture at 0x10edd2390 state=running>

Sleeping for : Sleeping for :  Sleeping for :  1 
11



In [4]:
dfkl = dag_vis.DFKListener(dfk)

FailFail
<AppFuture at 0x10f34b828 state=finished raised DependencyError>
Sleeping for : 
 <AppFuture at 0x10f34bb70 state=finished raised DependencyError>Fail
1
Sleeping for : 
<AppFuture at 0x10f34bc18 state=finished raised DependencyError> 
1Sleeping for : 
 1


In [5]:
dfkl.auto_updater(10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FailSucceed
FailSleeping for : 

 <AppFuture at 0x10f34bb38 state=finished raised DependencyError><AppFuture at 0x10f34ba90 state=finished raised DependencyError>1


Sleeping for : Sleeping for :   11

FailFailFail


<AppFuture at 0x10f34b9b0 state=finished raised DependencyError><AppFuture at 0x10ff3f2b0 state=finished raised DependencyError><AppFuture at 0x10ff3f358 state=finished raised DependencyError>


Sleeping for : Sleeping for :   01

Succeed
Fail
<AppFuture at 0x10ff3f048 state=finished raised DependencyError>
<AppFuture at 0x10f34b710 state=finished raised DependencyError>


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>